# Imports & Constants

In [1]:
from __future__ import division
import numpy as np
import sympy
sympy.init_printing()
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact
from scipy.integrate import odeint
from scipy.optimize import fsolve

In [2]:
g = 9.807
P_atm = 82 #kPa

# Fluid properties

In [3]:
ρ = 1127 #kg/m^3
SG = ρ/1000
μ = 0.012 #Pa.s

# Pipeline properties

In [4]:
d = 95.5 #in mm
d_inch = d * 0.03937 #in inches
D = d/1000 #in m
epsilon = 0.015 #in mm
W_n = 35000/60/60 #kg/s
W_d = 1.15*W_n #kg/s
W_m = 1.15/1.05 * W_n

# Suction line

In [5]:
length_suction = 2
dZ_suction = -8.2
resist_suction = [4, 0, 2, 0, 2, 0, 1, 0]
# (90 deg elbows, Check Valve, Butterfly valves, Gate Valves, soft tee, hard tee, 
# Pipe enterances, Pipe exits)

# Delivery line

In [6]:
length_delivery = 32.2
dZ_delivery = 10
resist_delivery = [24, 0, 12, 0, 21, 0, 0, 1]
# (90 deg elbows, Check Valve, Butterfly valves, Gate Valves, soft tee, hard tee, 
# Pipe enterances, Pipe exits)

# Functions

In [7]:
def reynolds(u):
    Re = (D * u * ρ) / μ
    return Re

def dPelevation(dh):
    dPel = ρ * g * dh / 1000
    return dPel

def dPeq_f(W):
    dPeqr = 52.90 + 68.06 + 72.77 + 55.40 + 1.73
    Wr = 30000/60/60
    dPeq = dPeqr * (W/Wr)**2
    return dPeq

def dP_friction(Lp, u, resistances):
    # Resistances should be given as
    # (90 deg elbows, Check Valve, Butterfly valves, Gate&Ball Valves, soft tee, hard tee, 
    # Pipe enterances, Pipe exits)
    # Valves are taken as fully open
    # All connections are taken as flanged
    K1 = [800, 2000, 800, 300, 150, 800, 160, 0]
    Kinf = [0.25, 10, 0.25, 0.1, 0.5, 0.8, 0.5, 1]
    # Initial Guess for f_prime
    sqrt_f_prime = 0.1414
    Re = reynolds(u)
    # Iterate to find f_prime using colebrook
    difference = 1
    while abs(difference) > 1E-5:
        sqrt_f_prime = 1/(-2 * np.log10(epsilon/D/3.7 + 2.51/(Re*sqrt_f_prime)))
        difference = 1/(sqrt_f_prime) - (-2 * np.log10(epsilon/D/3.7 + 2.51/(Re*sqrt_f_prime)))
    f_prime = sqrt_f_prime**2
    # Calculate the resistance coefficient for resistances
    Ktot = 0
    for i in range(0, 8, 1):
        if i < 6:
            K = K1[i]/Re + Kinf[i]*(1 + 1/d_inch)
            Ktot += K
        else:
            K = K1[i]/Re + Kinf[i]
            Ktot += K
    # Add Resistance for the pipe
    K_pipe = f_prime*Lp/D
    Ktot += K_pipe
    # Calculate friction loss
    dP_friction = Ktot*(ρ*u**2/2000)
    return dP_friction

def suction(W):
    # Mass flow rate in kg/s
    u = (W/ρ) / ((np.pi*D**2)/4)
    dPel = dPelevation(dZ_suction)
    dPke = 1*ρ/2000*(u**2)
    dPf = dP_friction(length_suction, u, resist_suction)
    P_suction_flange = -(dPel + dPke + dPf)
    return P_suction_flange

def delivery(W):
    # Mass flow rate in kg/s
    u = (W/ρ) / ((np.pi*D**2)/4)
    dPel = dPelevation(dZ_delivery)
    dPke = 1*ρ/2000*(-u**2)
    dPf = dP_friction(length_delivery, u, resist_delivery)
    dPeq = dPeq_f(W)
    P_delivery_flange = dPel + dPke + dPf + dPeq
    return P_delivery_flange


# Calculations
## Design conditions

In [8]:
P_delivery_flange = delivery(W_d) + P_atm
P_suction_flange = suction(W_d) + P_atm

dPa = P_delivery_flange-P_suction_flange

print("The pressure at the delivery flange is ", str(P_delivery_flange), "kPa abs")
print("The pressure at the suction flange is ", str(P_suction_flange), "kPa abs")
print("The pressure supplied by the pump is ", str(dPa), "kPa")

The pressure at the delivery flange is  710.142709182 kPa abs
The pressure at the suction flange is  150.178096328 kPa abs
The pressure supplied by the pump is  559.964612854 kPa


## Nominal Conditions

In [9]:
P_delivery_flange = delivery(W_n) + P_atm
P_suction_flange = suction(W_n) + P_atm

dPa = P_delivery_flange-P_suction_flange

print("The pressure at the delivery flange is ", str(P_delivery_flange), "kPa abs")
print("The pressure at the suction flange is ", str(P_suction_flange), "kPa abs")
print("The pressure supplied by the pump is ", str(dPa), "kPa")

The pressure at the delivery flange is  584.012735344 kPa abs
The pressure at the suction flange is  155.600997163 kPa abs
The pressure supplied by the pump is  428.411738181 kPa


## Maximum flow Conditions

In [10]:
P_delivery_flange = delivery(W_m) + P_atm
P_suction_flange = suction(W_m) + P_atm

dPa = P_delivery_flange-P_suction_flange

print("The pressure at the delivery flange is ", str(P_delivery_flange), "kPa abs")
print("The pressure at the suction flange is ", str(P_suction_flange), "kPa abs")
print("The pressure supplied by the pump is ", str(dPa), "kPa")

The pressure at the delivery flange is  662.057267653 kPa abs
The pressure at the suction flange is  152.244611848 kPa abs
The pressure supplied by the pump is  509.812655805 kPa


In [22]:
print (W_d, W_n, W_m)

11.180555555555555 9.722222222222223 10.648148148148149
